## 🚀 SetUp

### 🗂️ Google Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 📚 Library

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Library4Recommend
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics.pairwise import cosine_similarity

## 📦 Data

In [3]:
df1 = pd.read_csv("/content/drive/MyDrive/ComInKo/data/lyrics_with_levels.csv", encoding="cp949")

## 🤔 Feature Select   
### PCA를 활용한 특징 선택 방법

PCA(주성분 분석)를 이용한 특징 선택 과정은 데이터를 압축하면서 중요한 정보를 유지하려는 과정입니다. 아래는 주요 단계별 설명입니다.

---

#### 1. **숫자형 데이터만 선택**
- `select_dtypes()`를 사용해 숫자형 데이터(`float64`, `int64`)만 추출합니다.
- 분석에 필요하지 않은 열(`difficulty_level`)은 제거합니다.

**이유:**  
PCA는 숫자형 데이터에만 작동하며, 불필요한 열은 제거해야 결과가 깔끔합니다.

---

#### 2. **데이터 표준화**
- `StandardScaler`를 사용해 데이터의 각 열을 **평균 0, 표준편차 1로 변환**합니다.

**이유:**  
PCA는 데이터의 스케일에 영향을 받습니다. 스케일이 큰 열이 영향을 과대평가받는 것을 방지하기 위함입니다.

---

#### 3. **PCA 수행**
- PCA로 데이터를 **주성분(Principal Components)**으로 변환합니다.
- 주요 결과:
  - **로드 행렬 (Loadings):**  
    각 주성분이 원래 데이터의 열들에 대해 얼마나 영향을 받았는지를 나타냅니다.
  - **설명 분산 비율 (Explained Variance Ratio):**  
    각 주성분이 원래 데이터의 변동성을 얼마나 설명하는지를 나타냅니다.

---

#### 4. **로드 행렬을 활용한 주요 피처 선택**
- 로드 행렬에서 **절대값이 임계값(0.7)**을 넘는 열(특징)을 선택합니다.
  - 예: PC1에서 \( |계수| > 0.7 \)인 열은 해당 주성분의 주요 기여자입니다.

**이유:**  
주성분을 통해 원래 데이터에서 어떤 특징이 중요한지 알 수 있습니다.

---

#### 5. **설명 분산 비율을 활용한 주요 주성분 선택**
- 설명 분산 비율의 **누적 합**을 계산해, 누적 80% 이하의 분산을 설명하는 주성분만 선택합니다.
  - 예: PC1, PC2, PC3이 누적 80% 이하의 변동성을 설명한다면, 이들만 선택.

**이유:**  
중요한 주성분만 남기고 나머지는 제거해 데이터 차원을 줄입니다.

---

#### 결과
1. **중요한 원래 피처(열):**  
   로드 행렬을 통해 주요 특징을 선택.
2. **중요한 주성분:**  
   설명 분산 비율을 기준으로 정보량이 많은 주성분만 선택.

---

##### 요약
이 방법은 데이터의 차원을 줄이면서도 중요한 정보를 보존할 수 있는 효과적인 도구입니다. 이후 머신러닝 모델 학습 시 **계산 비용 감소**와 **성능 향상**에 도움을 줍니다.


In [4]:
# 샘플 데이터 생성
# 1. 숫자형 데이터 선택
df = df1.select_dtypes(include=["float64", "int64"])
df = df.drop(columns=["difficulty_level"])

# 1. 데이터 표준화
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

# 2. PCA 수행
pca = PCA()
pca.fit(scaled_data)

# 3. 로드 행렬 출력
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f"PC{i+1}" for i in range(len(pca.components_))],
    index=df.columns
)
explained_variance = pca.explained_variance_ratio_

print("PCA Loadings:\n", loadings)
print("Explained Variance Ratio:\n", explained_variance)

# 4. 주요 피처 선택
# 로드 값 기준 주요 피처 선택
threshold = 0.7  # 임계값 0.7 :
#
selected_features = loadings[
    (loadings.abs() > threshold).any(axis=1)  # 어느 주성분에서든 임계값 초과
].index.tolist()

print("\nSelected Features Based on Loadings:\n", selected_features)

# 5. 설명 분산 비율 기반 주요 주성분 확인
cumulative_variance = explained_variance.cumsum()  # 누적 설명 분산
important_pcs = [f"PC{i+1}" for i, var in enumerate(cumulative_variance) if var <= 0.8]  # 누적 80% 이하 주성분 선택
print("\nImportant Principal Components (<= 80% Variance):\n", important_pcs)

PCA Loadings:
                         PC1       PC2       PC3       PC4       PC5       PC6  \
popularity         0.229034 -0.303918  0.228395 -0.152659  0.031799 -0.536774   
acousticness      -0.256539 -0.329858 -0.094880  0.020135 -0.265436  0.035917   
danceability       0.270624 -0.050235 -0.367853 -0.303327 -0.097299 -0.140502   
energy             0.273308  0.453011  0.037977 -0.049766  0.170920 -0.020069   
loudness_db        0.199177  0.507065  0.163052 -0.106618  0.236486  0.071045   
vocalness          0.138745  0.152312 -0.330373  0.463596 -0.471642 -0.195994   
tempo_bpm         -0.013689  0.241038  0.581297  0.407679 -0.380074 -0.186989   
release_date_int   0.191885 -0.120952  0.351405 -0.244542 -0.285559  0.577037   
lyrics_length      0.310055 -0.105395 -0.170245  0.378641  0.206183  0.106328   
duration_ms       -0.256260 -0.000869 -0.035256  0.314644  0.503763 -0.076818   
valence            0.259249  0.243982 -0.281289 -0.207691 -0.245632 -0.079631   
artist_popula

## ⚙️ Modeling


In [6]:
selected_columns = ['energy','english_ratio', 'korean_ratio', 'dps']

# 2. 데이터 정리
# 결측값 처리 (평균값으로 대체)
df1[selected_columns] = df1[selected_columns].fillna(df1[selected_columns].mean())

# 데이터 정규화
scaler = RobustScaler()
scaled_features = scaler.fit_transform(df1[selected_columns])

# 코사인 유사도 계산
cosine_sim_matrix = cosine_similarity(scaled_features)

# 코사인 유사도를 데이터프레임으로 변환
similarity_df = pd.DataFrame(cosine_sim_matrix, index=df1['title'], columns=df1['title'])

# 3. 추천 함수
def recommend_songs(user_title, user_level, user_genre, df1, similarity_df, top_n=5):
    """
    사용자 입력에 따라 곡 추천 함수.
    Args:
    - user_title (str): 사용자가 입력한 곡 제목
    - user_level (str): 사용자의 한국어 수준
    - user_genre (str): 사용자의 선호 장르
    - df1 (pd.DataFrame): 원본 데이터프레임
    - similarity_df (pd.DataFrame): 코사인 유사도 행렬
    - top_n (int): 추천 곡 개수
    """
    if user_title not in similarity_df.index:
        return "입력한 곡이 데이터에 없습니다."

    # 난이도와 장르로 필터링
    filtered_df = df1[(df1['difficulty_level_category'] == user_level) &
                      (df1['specific_genres'] == user_genre)]
    if filtered_df.empty:
        return f"'{user_level}' 수준과 '{user_genre}' 장르에 해당하는 곡이 없습니다."

    # 유사도 기반 추천
    similar_songs = similarity_df[user_title].loc[filtered_df['title']].sort_values(ascending=False)
    recommendations = similar_songs.iloc[1:top_n+1]
    return recommendations

# 4. 레이더 차트 개별 시각화 함수
def visualize_recommendations_with_input(user_title, recommendations, df1, selected_columns):
    """
    추천 곡과 입력 곡의 특성을 개별 레이더 차트로 시각화.
    Args:
    - user_title (str): 사용자가 입력한 곡 제목
    - recommendations (pd.Series): 추천 곡 리스트 및 유사도 점수
    - df1 (pd.DataFrame): 원본 데이터프레임
    - selected_columns (list): 특성 컬럼 리스트
    """
    # 입력 곡 데이터 가져오기
    input_song_data = df1[df1['title'] == user_title][selected_columns]
    if input_song_data.empty:
        print(f"입력한 곡 '{user_title}'의 데이터를 찾을 수 없습니다.")
        return

    # 추천 곡 데이터 가져오기
    recommended_titles = recommendations.index.tolist()
    data = df1[df1['title'].isin(recommended_titles)][selected_columns]

    labels = selected_columns
    num_vars = len(labels)
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]  # 닫힌 도형을 만들기 위해 첫 번째 값을 추가

    # 입력 곡 레이더 차트 생성
    for idx, row in input_song_data.iterrows():
        values = row.tolist()
        values += values[:1]

        fig, ax = plt.subplots(figsize=(8, 6), subplot_kw=dict(polar=True))
        ax.plot(angles, values, label=f"Input Song: {user_title}", linewidth=2, color='red')
        ax.fill(angles, values, alpha=0.25, color='red')

        ax.set_yticks([])
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels, fontsize=10)
        ax.set_title(f"Radar Chart for Input Song: '{user_title}'", fontsize=14)
        ax.legend(loc='upper right', fontsize=10)
        plt.show()

    # 각 추천 곡에 대해 개별 레이더 차트 생성
    for title, row in zip(recommended_titles, data.iterrows()):
        values = row[1].tolist()
        values += values[:1]

        fig, ax = plt.subplots(figsize=(8, 6), subplot_kw=dict(polar=True))
        ax.plot(angles, values, label=title, linewidth=2)
        ax.fill(angles, values, alpha=0.25)

        ax.set_yticks([])
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels, fontsize=10)
        ax.set_title(f"Radar Chart for '{title}'", fontsize=14)
        ax.legend(loc='upper right', fontsize=10)
        plt.show()

# 5. 사용자 입력
user_level = input("본인의 한국어 수준을 입력하세요 (초급, 중급, 고급 중 하나): ").strip()
user_genre = input("좋아하는 장르를 입력해주세요 (예: k-pop, 발라드, 힙합, 인디): ").strip()
user_title = input("좋아하는 노래 제목을 입력해주세요: ").strip()

# 추천 함수 실행 및 레이더 차트 시각화
recommendations = recommend_songs(user_title, user_level, user_genre, df1, similarity_df, top_n=5)

if isinstance(recommendations, pd.Series):
    print(f"\n'{user_title}'와 비슷한 '{user_level}' 수준의 '{user_genre}' 장르 추천 곡들:")
    print(recommendations)
    visualize_recommendations_with_input(user_title, recommendations, df1, selected_columns)
else:
    print(recommendations)


본인의 한국어 수준을 입력하세요 (초급, 중급, 고급 중 하나): 고급
좋아하는 장르를 입력해주세요 (예: k-pop, 발라드, 힙합, 인디): k-pop
좋아하는 노래 제목을 입력해주세요: fantasitic baby
입력한 곡이 데이터에 없습니다.
